In [7]:
file_path = "results/experiment_results_20260107_185301.csv"
import pandas as pd
data = pd.read_csv(file_path)

In [8]:
# display me if when ALG_algorithm is 'probSAT'
data[data['ALG_algorithm'] == 'probSAT']

,ALG_algorithm,ALG_max_flips,ALG_max_tries,ALG_seed,INST_n_variables,INST_instance_id,OUT_flips_count,OUT_clause_satisfied,OUT_clause_total,OUT_satisfiable
1,probSAT,2000,30,655441,20,1,42,84,84,True
3,probSAT,2000,30,390835,20,1,16,84,84,True
5,probSAT,2000,30,558896,20,1,18,84,84,True
7,probSAT,2000,30,808987,20,1,125,84,84,True
9,probSAT,2000,30,673869,20,1,57,84,84,True
...,...,...,...,...,...,...,...,...,...,...
151,probSAT,7500,30,326613,75,2,3356,315,315,True
153,probSAT,7500,30,280405,75,2,1764,315,315,True
155,probSAT,7500,30,597987,75,2,131,315,315,True
157,probSAT,7500,30,167745,75,2,54,315,315,True


In [10]:
# display if OUT_flips_count is greater then ALG_max_flips
data[data['OUT_flips_count'] == (data['ALG_max_flips'] * data['ALG_max_tries'])]

,ALG_algorithm,ALG_max_flips,ALG_max_tries,ALG_seed,INST_n_variables,INST_instance_id,OUT_flips_count,OUT_clause_satisfied,OUT_clause_total,OUT_satisfiable
100,GSAT,6000,30,946463,60,2,180000,247,252,False
101,probSAT,6000,30,946463,60,2,180000,251,252,False
102,GSAT,6000,30,247102,60,2,180000,251,252,False
103,probSAT,6000,30,247102,60,2,180000,251,252,False
104,GSAT,6000,30,192944,60,2,180000,251,252,False
105,probSAT,6000,30,192944,60,2,180000,251,252,False
106,GSAT,6000,30,383677,60,2,180000,248,252,False
107,probSAT,6000,30,383677,60,2,180000,251,252,False
108,GSAT,6000,30,359755,60,2,180000,251,252,False
109,probSAT,6000,30,359755,60,2,180000,251,252,False


In [6]:
data[data['OUT_flips_count'] == data['ALG_max_flips']]

,ALG_algorithm,ALG_max_flips,ALG_max_tries,ALG_seed,INST_n_variables,INST_instance_id,OUT_flips_count,OUT_clause_satisfied,OUT_clause_total,OUT_satisfiable
101,probSAT,6000,NaN,211876,60,2,6000,251,252,False
103,probSAT,6000,NaN,218278,60,2,6000,251,252,False
105,probSAT,6000,NaN,253275,60,2,6000,251,252,False
107,probSAT,6000,NaN,96764,60,2,6000,251,252,False
109,probSAT,6000,NaN,947234,60,2,6000,251,252,False
111,probSAT,6000,NaN,656935,60,2,6000,251,252,False
113,probSAT,6000,NaN,726377,60,2,6000,251,252,False
115,probSAT,6000,NaN,717171,60,2,6000,251,252,False
117,probSAT,6000,NaN,165054,60,2,6000,251,252,False
119,probSAT,6000,NaN,193459,60,2,6000,251,252,False
